In [1]:
% matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import pickle

from datetime import datetime, date, time
from textblob import TextBlob
from textblob import Word
from pre_processing import *
from nltk.corpus import stopwords
pd.set_option('display.max_colwidth', -1)

In [2]:
#Load Data Sets
DATA_FOLDER = './data'

tweets_col_names=['text']

tweets_dtypes = {'text': str }

print("Loading Data Set....")
tweets_neg = pd.read_fwf(DATA_FOLDER + '/train_neg.txt',  names=tweets_col_names,
                                  dtype=tweets_dtypes)
tweets_pos = pd.read_fwf(DATA_FOLDER + '/train_pos.txt',  names=tweets_col_names,
                                  dtype=tweets_dtypes)
tweets_neg_full = pd.read_fwf(DATA_FOLDER + '/train_neg_full.txt',  names=tweets_col_names,
                                  dtype=tweets_dtypes)
tweets_pos_full = pd.read_fwf(DATA_FOLDER + '/train_pos_full.txt',  names=tweets_col_names,
                                  dtype=tweets_dtypes)
tweets_test = pd.read_fwf(DATA_FOLDER + '/test_data.txt', header=None)
tweets_test.drop(columns=[1,2], inplace=True)
tweets_test.rename(columns={0:'text'}, inplace= True)

    
# if not cleaned_test:  
#     #read data from .txt
#     test_data = pd.read_fwf(DATA_FOLDER + '/test_data.txt', )
#     #index start at 1
#     test_data.index = test_data.index +1
#     #remove duplicates
#     test_data.drop_duplicates(inplace=True)
#     test_data.drop(columns=[1,2], inplace=True)
#     test_data.rename(columns={0:'text'}, inplace= True)
print("Data Set loaded !")


#bool variables to clean data
#If set to False the cleaning process will take place
already_cleaned_neg = False
already_cleaned_pos = False
already_cleaned_neg_full = True
already_cleaned_pos_full = True
already_cleaned_test = False

#variable to define if saved the lemmatized text or not, if set to false will save column with no lemmatize
lemmatize = False


# freq_word = pd.read_fwf(DATA_FOLDER + '/pre_processed/freq_words_10_withoutpronouns.txt',  names=tweets_col_names,
#                                   dtype=tweets_dtypes)
# freq_word = list(freq_word.text)
# # freq_word

Loading Data Set....
Data Set loaded !


In [3]:
#read data from .txt
test_data = pd.read_fwf(DATA_FOLDER + '/test_data.txt', header=None)
#index start at 1
test_data.index = test_data.index +1
#remove duplicates
test_data.drop_duplicates(inplace=True)
test_data.drop(columns=[1,2], inplace=True)
test_data.rename(columns={0:'text'}, inplace= True)
    

In [4]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
stop = stopwords.words('english')
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]


In [5]:
def clean_data(data):
    """Function to clean the data.

    Args:
        data: data to clean
        
    Returns:
        data cleaned
    """
    #increase index to start at 1
    data.index = data.index +1
    
    #remove duplicates
    data.drop_duplicates(inplace=True)
    
    #apply pre_process funtioncs
    print("emoji")
    data['text'] = data['text'].apply(lambda x: interpret_emoji(x))
    
    print("hastg!!")
    #remove hashtag #
    data['text'] = data['text'].str.replace('#', '')
    
    print("contraction!!")
    #expand contractions don't => do not
    data['text'] = data['text'].apply(lambda x: expand_contractions(x))
    
    print("special cases!!")
        #replace ur by your and other special cases
    data['text'] = data['text'].str.replace(' ur ', ' your ')
    data['text'] = data['text'].str.replace(' u ', ' you ')
    data['text'] = data['text'].str.replace(' cant ', ' can not ')
    data['text'] = data['text'].str.replace(' yourl ', ' your ')
    data['text'] = data['text'].str.replace(' lol ', ' laugh ')
    
    print("numbers!!")
    #separate number with letters  1234test123 =>  test 
    data['text'] = data['text'].apply(lambda x: split_number_text(x))
   
    print("punctuaction!!")
    #remove punctuaction .........
    data['text'] = data['text'].apply(lambda x: remove_punctuation(x))
    
    print("user,url,number!!")
    # remove words user, url, number
    data['text'] = data['text'].apply(lambda x: remove_words(x))
    
    print("more letters!")
    # replace more letters haaaaaaaaaappy => haappy
    data['text'] = data['text'].apply(lambda x: replace_moreletters(x))
    
    print("lemmatizing!")
        #lemmatize words
    data['text_lema'] = data['text'].apply(lemmatize_text)
    data['text_lema'] = data['text_lema'].apply(lambda x: ' '.join(x))
    data['text_lema'] = data['text_lema'].str.replace(',','')
    return  data



In [6]:
if not already_cleaned_neg :
    print("Cleaning Tweets Neg")
    clean_data(tweets_neg).head()
    #save the file to pickle
    print("Saving file with preprocessed Tweets")
    pickle_out = open("data/pre_processed/tweets_neg_no_textblob.pickle","wb")
    if not lemmatize:
        pickle.dump(tweets_neg['text'], pickle_out)
    else:
        pickle.dump(tweets_neg['text_lema'], pickle_out)
    pickle_out.close()
    #save file to text
    f = open("data/pre_processed/tweets_neg_no_textblob.txt", "w", encoding='utf-8')
    if not lemmatize:
        f.write("\n".join(map(lambda x: str(x), tweets_neg['text'])))
    else:
        f.write("\n".join(map(lambda x: str(x), tweets_neg['text_lema'])))
    f.close()
    print("Saved!")
else:
    print("Tweets Pos Already Cleaned!")
pickle_in = open("data/pre_processed/tweets_neg_no_textblob.pickle","rb")
tweets_neg_pickle = pickle.load(pickle_in)
print("Opening pickle")
tweets_neg_pickle.head()


Cleaning Tweets Neg
emoji
hastg!!
contraction!!
special cases!!
numbers!!
punctuaction!!
user,url,number!!
more letters!
lemmatizing!
Saving file with preprocessed Tweets
Saved!
Opening pickle


1    vinco tresorpack difficulty of object disassemble and reassemble the wooden pieces this beautiful wo
2    glad dot have taks tomorrow thankful startho                                                        
3    vs celtics in the regular season were fucked if we play them in the playoffs                        
4    could actually kill that girl am so sorry                                                           
5    find that very hard to believe im afraid                                                            
Name: text, dtype: object

In [ ]:
if not already_cleaned_pos :
    print("Cleaning Tweets Pos")
    clean_data(tweets_pos).head()
    #save the file to pickle
    print("Saving file with preprocessed Tweets")
    pickle_out = open("data/pre_processed/tweets_pos_no_textblob.pickle","wb")
    if not lemmatize:
        pickle.dump(tweets_pos['text'], pickle_out)
    else:
        pickle.dump(tweets_pos['text_lema'], pickle_out)
    pickle_out.close()
    #save file to text
    f = open("data/pre_processed/tweets_pos_no_textblob.txt", "w", encoding='utf-8')
    if not lemmatize:
        f.write("\n".join(map(lambda x: str(x), tweets_pos['text'])))
    else:
        f.write("\n".join(map(lambda x: str(x), tweets_pos['text_lema'])))
    f.close()
    print("Saved!")
else:
    print("Tweets Neg Already Cleaned!")
pickle_in = open("data/pre_processed/tweets_pos_no_textblob.pickle","rb")
tweets_pos_pickle = pickle.load(pickle_in)
print("Opening pickle")
tweets_pos_pickle.head()
    

Cleaning Tweets Pos
emoji


In [ ]:
# if not already_cleaned_neg_full :
#     print("Cleaning Tweets Neg Full")
#     clean_data(tweets_neg_full).head()
#     #save the file to pickle
#     print("Saving file with preprocessed Tweets")
#     pickle_out = open("data/pre_processed/tweets_neg_full_no_textblob.pickle","wb")
#     if not lemmatize:
#         pickle.dump(tweets_neg_full['text'], pickle_out)
#     else:
#         pickle.dump(tweets_neg_full['text_lema'], pickle_out)
#     pickle_out.close()
#     #save file to text
#     f = open("data/pre_processed/tweets_neg_full_no_textblob.txt", "w", encoding='utf-8')
#     if not lemmatize:
#         f.write("\n".join(map(lambda x: str(x), tweets_neg_full['text'])))
#     else:
#         f.write("\n".join(map(lambda x: str(x), tweets_neg_full['text_lema'])))
#     f.close()
#     print("Saved!")
# else:
#     print("Tweets Neg Full Already Cleaned!")
# pickle_in = open("data/pre_processed/tweets_neg_full_no_textblob.pickle","rb")
# tweets_neg_full_pickle = pickle.load(pickle_in)
# print("Opening pickle")
# tweets_neg_full_pickle.head()
    

In [ ]:
# if not already_cleaned_pos_full :
#     print("Cleaning Tweets Pos Full")
#     clean_data(tweets_pos_full).head()
#     #save the file to pickle
#     print("Saving file with preprocessed Tweets")
#     pickle_out = open("data/pre_processed/tweets_pos_full_no_textblob.pickle","wb")
#     if not lemmatize:
#         pickle.dump(tweets_pos_full['text'], pickle_out)
#     else:
#         pickle.dump(tweets_pos_full['text_lema'], pickle_out)
#     pickle_out.close()
#     #save file to text
#     f = open("data/pre_processed/tweets_pos_full_no_textblob.txt", "w", encoding='utf-8')
#     if not lemmatize:
#         f.write("\n".join(map(lambda x: str(x), tweets_pos_full['text'])))
#     else:
#         f.write("\n".join(map(lambda x: str(x), tweets_pos_full['text_lema'])))
#     f.close()
#     print("Saved!")
# else:
#     print("Tweets Pos Full Already Cleaned!")
# pickle_in = open("data/pre_processed/tweets_pos_full_no_textblob.pickle","rb")
# tweets_pos_full_pickle = pickle.load(pickle_in)
# print("Opening pickle")
# tweets_pos_full_pickle.head()
    

In [ ]:
if not already_cleaned_test :
    tweets_test.index = tweets_test.index +1
    print("Cleaning Test")
    clean_data(tweets_test).head()
    print("Saving file with preprocessed Tweets")
    #save to pickle
    pickle_out = open("data/pre_processed/test_data_no_textblob.pickle","wb")
    if not lemmatize:
        pickle.dump(tweets_test['text'], pickle_out)
    else:
        pickle.dump(tweets_test['text_lema'], pickle_out)
    pickle_out.close()
    #save to txt
    f = open("data/pre_processed/test_data_no_textblob.txt", "w", encoding='utf-8')
    if not lemmatize:
        f.write("\n".join(map(lambda x: str(x[0]+1) + ',' + str(x[1]), enumerate(tweets_test['text']))))
    else:
        f.write("\n".join(map(lambda x: str(x[0]+1) + ',' + str(x[1]), enumerate(tweets_test['text_lema']))))
    f.close()
    print("Saved!")
else:
    print("Test Already Cleaned!")
pickle_in = open("data/pre_processed/test_data_no_textblob.pickle","rb")
tweets_test_pickle = pickle.load(pickle_in)
print("Opening pickle")
tweets_test_pickle.head()
    

In [ ]:
# #verify  flag for data already cleaned
# if not cleaned_neg :
#     #read data from .txt to preprocess
#     tweets_neg = pd.read_fwf(DATA_FOLDER + '/train_neg.txt',  names=tweets_col_names,
#                                   dtype=tweets_dtypes)
    
#     #increase index to start at 1
#     tweets_neg.index = tweets_neg.index +1
#     #remove duplicates
#     tweets_neg.drop_duplicates(inplace=True)
#     test = tweets_neg.head(50)
#     #apply pre_process funtioncs
#     print("emoji")
#     # interpret emoji
#     tweets_neg['text'] = tweets_neg['text'].apply(lambda x: interpret_emoji(x))
#     print("numbers!!")
#     #separate number with letters  1234test123 =>  test 
#     tweets_neg['text'] = tweets_neg['text'].apply(lambda x: split_number_text(x))
#     print("hastg!!")
#     #remove hashtag #
#     tweets_neg['text'] = tweets_neg['text'].str.replace('#', '')
#     print("contraction!!")
#     #expand contractions don't => do not
#     tweets_neg['text'] = tweets_neg['text'].apply(lambda x: expand_contractions(x))
#     print("special cases!!")
#         #replace ur by your and other special cases
#     tweets_neg['text'] = tweets_neg['text'].str.replace(' ur ', ' your ')
#     tweets_neg['text'] = tweets_neg['text'].str.replace(' u ', ' you ')
#     tweets_neg['text'] = tweets_neg['text'].str.replace(' cant ', ' can not ')
#     tweets_neg['text'] = tweets_neg['text'].str.replace(' yourl ', ' your ')
#     tweets_neg['text'] = tweets_neg['text'].str.replace(' lol ', ' laugh ')
#     print("punctuaction!!")
#     #remove punctuaction .........
#     tweets_neg['text'] = tweets_neg['text'].apply(lambda x: remove_punctuation(x))
#     print("user,url,number!!")
#     # remove words user, url, number
#     tweets_neg['text'] = tweets_neg['text'].apply(lambda x: remove_words(x))
#     print("more letters!")
#     # replace more letters haaaaaaaaaappy => haappy
#     tweets_neg['text'] = tweets_neg['text'].apply(lambda x: replace_moreletters(x))

#     #correct words with textblob
# #     print("TEXT BLOB!!")
# #     tweets_neg['text'] = tweets_neg['text'].apply(lambda x: ''.join(TextBlob(x).correct()))
#     # tweets_neg
    
# #     #lemmatize words
# #     tweets_neg['text_lema'] = tweets_neg['text'].apply(lemmatize_text)
    
# #     tweets_neg['text_lema'] = tweets_neg['text_lema'].apply(lambda x: ' '.join(x))
# #     tweets_neg['text_lema'] = tweets_neg['text_lema'].str.replace(',','')
# # #     tweets_neg['text_lema'] = tweets_neg['text_lema'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_words_10))
   
#     #save the file to pickle
#     print("Saving file with preprocessed Tweets")
#     pickle_out = open("data/pre_processed/tweets_neg_no_textblob.pickle","wb")
#     pickle.dump(tweets_neg['text'], pickle_out)
#     pickle_out.close()
#     f = open("data/pre_processed/tweets_neg_no_textblob.txt", "w", encoding='utf-8')
#     f.write("\n".join(map(lambda x: str(x), tweets_neg['text'])))
#     f.close()
#     print("Saved!")

# else:
#     print("Already Trained!")
    
# pickle_in = open("data/pre_processed/tweets_neg_no_textblob.pickle","rb")
# tweets_neg_pickle = pickle.load(pickle_in)
# print("Opening pickle")
# tweets_neg_pickle.head()
    

In [ ]:
# if not cleaned_pos:
#     #read data .txt file
#     tweets_pos = pd.read_fwf(DATA_FOLDER + '/train_pos.txt',  names=tweets_col_names,
#                                   dtype=tweets_dtypes)
#     # index start at 1
#     tweets_pos.index = tweets_pos.index + 1
#     #remove duplicates
#     tweets_pos.drop_duplicates(inplace=True)
    
#     #apply pre_process funtioncs
#     # interpret emoji
#     print("emoji")
#     # interpret emoji
#     tweets_pos['text'] = tweets_pos['text'].apply(lambda x: interpret_emoji(x))
#     print("numbers!!")
#     #separate number with letters  1234test123 =>  test 
#     tweets_pos['text'] = tweets_pos['text'].apply(lambda x: split_number_text(x))
#     print("hastg!!")
#     #remove hashtag #
#     tweets_pos['text'] = tweets_pos['text'].str.replace('#', '')
#     print("contraction!!")
#     #expand contractions don't => do not
#     tweets_pos['text'] = tweets_pos['text'].apply(lambda x: expand_contractions(x))
#     print("special cases!!")
#         #replace ur by your and other special cases
#     tweets_pos['text'] = tweets_pos['text'].str.replace(' ur ', ' your ')
#     tweets_pos['text'] = tweets_pos['text'].str.replace(' u ', ' you ')
#     tweets_pos['text'] = tweets_pos['text'].str.replace(' cant ', ' can not ')
#     tweets_pos['text'] = tweets_pos['text'].str.replace(' yourl ', ' your ')
#     tweets_pos['text'] = tweets_pos['text'].str.replace(' lol ', ' laugh ')
#     print("punctuaction!!")
#     #remove punctuaction .........
#     tweets_pos['text'] = tweets_pos['text'].apply(lambda x: remove_punctuation(x))
#     print("user,url,number!!")
#     # remove words user, url, number
#     tweets_pos['text'] = tweets_pos['text'].apply(lambda x: remove_words(x))
#     print("more letters!")
#     # replace more letters haaaaaaaaaappy => haappy
#     tweets_pos['text'] = tweets_pos['text'].apply(lambda x: replace_moreletters(x))

#     #correct words with textblob
# #     print("TEXT BLOB!!")
# #     tweets_pos['text'] = tweets_pos['text'].apply(lambda x: ''.join(TextBlob(x).correct()))
    
# #     #lemmatize words
# #     tweets_neg['text_lema'] = tweets_neg['text'].apply(lemmatize_text)
    
# #     tweets_neg['text_lema'] = tweets_neg['text_lema'].apply(lambda x: ' '.join(x))
# #     tweets_neg['text_lema'] = tweets_neg['text_lema'].str.replace(',','')
# # #     tweets_neg['text_lema'] = tweets_neg['text_lema'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_words_10))
   
#     #save the file to pickle
#     print("Saving file with preprocessed Tweets")
#     pickle_out = open("data/pre_processed/tweets_pos_no_textblob.pickle","wb")
#     pickle.dump(tweets_pos['text'], pickle_out)
#     pickle_out.close()
#     f = open("data/pre_processed/tweets_pos_no_textblob.txt", "w", encoding='utf-8')
#     f.write("\n".join(map(lambda x: str(x), tweets_pos['text'])))
#     f.close()
#     print("Saved!")

# else:
#     print("Already Trained!")
    
# pickle_in = open("data/pre_processed/tweets_pos_no_textblob.pickle","rb")
# tweets_pos_pickle = pickle.load(pickle_in)
# print("Opening pickle")
# tweets_pos_pickle.head()
    

In [ ]:
# #verify  flag for data already cleaned
# if not cleaned_neg_full :
#     #read data from .txt to preprocess
#     tweets_neg_full = pd.read_fwf(DATA_FOLDER + '/train_neg_full.txt',  names=tweets_col_names,
#                                   dtype=tweets_dtypes)
    
#     #increase index to start at 1
#     tweets_neg_full.index = tweets_neg_full.index +1
#     #remove duplicates
#     tweets_neg_full.drop_duplicates(inplace=True)
#     #apply pre_process funtioncs
#     print("emoji")
#     # interpret emoji
#     tweets_neg_full['text'] = tweets_neg_full['text'].apply(lambda x: interpret_emoji(x))
#     print("numbers!!")
#     #separate number with letters  1234test123 =>  test 
#     tweets_neg_full['text'] = tweets_neg_full['text'].apply(lambda x: split_number_text(x))
#     print("hastg!!")
#     #remove hashtag #
#     tweets_neg_full['text'] = tweets_neg_full['text'].str.replace('#', '')
#     print("contraction!!")
#     #expand contractions don't => do not
#     tweets_neg_full['text'] = tweets_neg_full['text'].apply(lambda x: expand_contractions(x))
#     print("special cases!!")
#         #replace ur by your and other special cases
#     tweets_neg_full['text'] = tweets_neg_full['text'].str.replace(' ur ', ' your ')
#     tweets_neg_full['text'] = tweets_neg_full['text'].str.replace(' u ', ' you ')
#     tweets_neg_full['text'] = tweets_neg_full['text'].str.replace(' cant ', ' can not ')
#     tweets_neg_full['text'] = tweets_neg_full['text'].str.replace(' yourl ', ' your ')
#     tweets_neg_full['text'] = tweets_neg_full['text'].str.replace(' lol ', ' laugh ')
#     print("punctuaction!!")
#     #remove punctuaction .........
#     tweets_neg_full['text'] = tweets_neg_full['text'].apply(lambda x: remove_punctuation(x))
#     print("user,url,number!!")
#     # remove words user, url, number
#     tweets_neg_full['text'] = tweets_neg_full['text'].apply(lambda x: remove_words(x))
#     print("more letters!")
#     # replace more letters haaaaaaaaaappy => haappy
#     tweets_neg_full['text'] = tweets_neg_full['text'].apply(lambda x: replace_moreletters(x))

#     #correct words with textblob
#     print("TEXT BLOB!!")
#     tweets_neg_full['text'] = tweets_neg_full['text'].apply(lambda x: ''.join(TextBlob(x).correct()))
    
# #     #lemmatize words
# #     tweets_neg_full['text_lema'] = tweets_neg_full['text'].apply(lemmatize_text)
    
# #     tweets_neg_full['text_lema'] = tweets_neg_full['text_lema'].apply(lambda x: ' '.join(x))
# #     tweets_neg_full['text_lema'] = tweets_neg_full['text_lema'].str.replace(',','')
   
#     #save the file to pickle
#     print("Saving file with preprocessed Tweets")
#     pickle_out = open("data/pre_processed/tweets_neg_full_textblob.pickle","wb")
#     pickle.dump(tweets_neg_full['text'], pickle_out)
#     pickle_out.close()
#     f = open("data/pre_processed/tweets_neg_full_textblob.txt", "w", encoding='utf-8')
#     f.write("\n".join(map(lambda x: str(x), tweets_neg_full['text'])))
#     f.close()
#     print("Saved!")

# else:
#     print("Already Trained!")
    
# pickle_in = open("data/pre_processed/tweets_neg_full_textblob.pickle","rb")
# tweets_neg_pickle = pickle.load(pickle_in)
# print("Opening pickle")
# tweets_neg_pickle.head()

In [ ]:
# if not cleaned_pos_full:
#     #read data .txt file
#     tweets_pos_full = pd.read_fwf(DATA_FOLDER + '/train_pos_full.txt',  names=tweets_col_names,
#                                   dtype=tweets_dtypes)
#     # index start at 1
#     tweets_pos_full.index = tweets_pos_full.index + 1
#     #remove duplicates
#     tweets_pos_full.drop_duplicates(inplace=True)
#     #apply pre_process funtioncs
#     print("emoji")
#     # interpret emoji
#     tweets_pos_full['text'] = tweets_pos_full['text'].apply(lambda x: interpret_emoji(x))
#     print("numbers!!")
#     #separate number with letters  1234test123 =>  test 
#     tweets_pos_full['text'] = tweets_pos_full['text'].apply(lambda x: split_number_text(x))
#     print("hastg!!")
#     #remove hashtag #
#     tweets_pos_full['text'] = tweets_pos_full['text'].str.replace('#', '')
#     print("contraction!!")
#     #expand contractions don't => do not
#     tweets_pos_full['text'] = tweets_pos_full['text'].apply(lambda x: expand_contractions(x))
#     print("special cases!!")
#         #replace ur by your and other special cases
#     tweets_pos_full['text'] = tweets_pos_full['text'].str.replace(' ur ', ' your ')
#     tweets_pos_full['text'] = tweets_pos_full['text'].str.replace(' u ', ' you ')
#     tweets_pos_full['text'] = tweets_pos_full['text'].str.replace(' cant ', ' can not ')
#     tweets_pos_full['text'] = tweets_pos_full['text'].str.replace(' yourl ', ' your ')
#     tweets_pos_full['text'] = tweets_pos_full['text'].str.replace(' lol ', ' laugh ')
#     print("punctuaction!!")
#     #remove punctuaction .........
#     tweets_pos_full['text'] = tweets_pos_full['text'].apply(lambda x: remove_punctuation(x))
#     print("user,url,number!!")
#     # remove words user, url, number
#     tweets_pos_full['text'] = tweets_pos_full['text'].apply(lambda x: remove_words(x))
#     print("more letters!")
#     # replace more letters haaaaaaaaaappy => haappy
#     tweets_pos_full['text'] = tweets_pos_full['text'].apply(lambda x: replace_moreletters(x))

#     #correct words with textblob
#     print("TEXT BLOB!!")
#     tweets_pos_full['text'] = tweets_pos_full['text'].apply(lambda x: ''.join(TextBlob(x).correct()))
#     # tweets_neg
    
# #     #lemmatize words
# #     tweets_pos_full['text_lema'] = tweets_pos_full['text'].apply(lemmatize_text)
    
# #     tweets_pos_full['text_lema'] = tweets_pos_full['text_lema'].apply(lambda x: ' '.join(x))
# #     tweets_pos_full['text_lema'] = tweets_pos_full['text_lema'].str.replace(',','')
   
#     #save the file to pickle
#     print("Saving file with preprocessed Tweets")
#     pickle_out = open("data/pre_processed/tweets_pos_full_textblob.pickle","wb")
#     pickle.dump(tweets_pos_full['text'], pickle_out)
#     pickle_out.close()
#     f = open("data/pre_processed/tweets_pos_full_textblob.txt", "w", encoding='utf-8')
#     f.write("\n".join(map(lambda x: str(x), tweets_pos_full['text'])))
#     f.close()
#     print("Saved!")

# else:
#     print("Already Trained!")
    
# pickle_in = open("data/pre_processed/tweets_pos_full_textblob.pickle","rb")
# tweets_neg_pickle = pickle.load(pickle_in)
# print("Opening pickle")
# tweets_neg_pickle.head()

In [ ]:
# if not cleaned_test:  
#     #read data from .txt
#     test_data = pd.read_fwf(DATA_FOLDER + '/test_data.txt', header=None)
#     #index start at 1
#     test_data.index = test_data.index +1
#     #remove duplicates
#     test_data.drop_duplicates(inplace=True)
#     test_data.drop(columns=[1,2], inplace=True)
#     test_data.rename(columns={0:'text'}, inplace= True)
    
#     #apply pre_process funtioncs
#     #remove stop words
# #     test_data['text'] = test_data['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
#     # interpret emoji
#     test_data['text'] = test_data['text'].apply(lambda x: interpret_emoji(x))
#     #separate number with letters  1234test123 => test 
#     test_data['text'] = test_data['text'].apply(lambda x: split_number_text(x))
#     #remove hashtag #
#     test_data['text'] = test_data['text'].str.replace('#', '')
#     #replace ur by your
#     test_data['text'] = test_data['text'].str.replace('ur', 'your')
#     #expand contractions don't => do not
#     test_data['text'] = test_data['text'].apply(lambda x: expand_contractions(x))
#     test_data['text'] = test_data['text'].str.replace(' ur ', ' ')
#     test_data['text'] = test_data['text'].str.replace(' u ', ' ')
#     test_data['text'] = test_data['text'].str.replace(' cant ', ' can not ')
#     test_data['text'] = test_data['text'].str.replace(' your ', ' ')
#     test_data['text'] = test_data['text'].str.replace(' yourl ', ' ')

#             #remove punctuaction
#     test_data['text'] = test_data['text'].apply(lambda x: remove_punctuation(x))
 
#     # remove words user, url, number
#     test_data['text'] = test_data['text'].apply(lambda x: remove_words(x))
#     # replace more letters haaaaaaaaaapy => haapy
#     test_data['text'] = test_data['text'].apply(lambda x: replace_moreletters(x))

#     #correct words with textblob
#     print("TEXT BLOB!!!")
#     tweets_pos['text'] = tweets_pos['text'].apply(lambda x: ''.join(TextBlob(x).correct()))
#     # tweets_pos
    

        
#     #lemmatize words
# #     test_data['text_lema'] = test_data['text'].apply(lemmatize_text)
    
    
# #     test_data['text_lema'] = test_data['text_lema'].apply(lambda x: ' '.join(x))
# #     test_data['text_lema'] = test_data['text_lema'].str.replace(',','')

#     print("Saving file with preprocessed Tweets")
#     pickle_out = open("data/pre_processed/test_data_no_textblob.pickle","wb")
#     pickle.dump(test_data['text'], pickle_out)
#     pickle_out.close()
#     f = open("data/pre_processed/test_data_textblob.txt", "w", encoding='utf-8')
#     f.write("\n".join(map(lambda x: str(x[0]+1) + ',' + str(x[1]), enumerate(test_data['text']))))
#     f.close()
#     print("Saved!")
# else:
#     print("Already Trained!")
# pickle_in = open("data/pre_processed/test_data_no_textblob.pickle","rb")
# test_data_pickle = pickle.load(pickle_in)
# print("Opening pickle")
# test_data_pickle.head()

In [ ]:

# #read data .txt file
# tweets_pos_full = pd.read_fwf(DATA_FOLDER + '/train_pos_full.txt',  names=tweets_col_names,
#                               dtype=tweets_dtypes)
# # index start at 1
# tweets_pos_full.index = tweets_pos_full.index + 1
# #remove duplicates
# tweets_pos_full.drop_duplicates(inplace=True)
# #apply pre_process funtioncs
# #remove stop words
# #     tweets_pos['text'] = tweets_pos['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
# print("Interpreting emojis!!")
#  # interpret emoji
# tweets_pos_full['text'] = tweets_pos_full['text'].apply(lambda x: interpret_emoji(x))
# print("Separating Numbers!!")
# #separate number with letters  1234test123 =>  test 
# tweets_pos_full['text'] = tweets_pos_full['text'].apply(lambda x: split_number_text(x))
# #     print(tweets_pos_full['text'].count())
# print("Remove Hashtags!!")
# #remove hashtag #
# tweets_pos_full['text'] = tweets_pos_full['text'].str.replace('#', '')
# print("Replace ur by your!!")
# #replace ur by your
# tweets_pos_full['text'] = tweets_pos_full['text'].str.replace('ur', 'your')
# print("expand contractions!!")
# #expand contractions don't => do not
# tweets_pos_full['text'] = tweets_pos_full['text'].apply(lambda x: expand_contractions(x))
# tweets_pos_full['text'] = tweets_pos_full['text'].str.replace(' ur ', '')
# tweets_pos_full['text'] = tweets_pos_full['text'].str.replace(' u ', '')
# tweets_pos_full['text'] = tweets_pos_full['text'].str.replace(' i ', '')
# tweets_pos_full['text'] = tweets_pos_full['text'].str.replace(' he ', '')
# tweets_pos_full['text'] = tweets_pos_full['text'].str.replace(' she ', '')
# tweets_pos_full['text'] = tweets_pos_full['text'].str.replace(' they ', '')
# tweets_pos_full['text'] = tweets_pos_full['text'].str.replace(' it ', '')
# tweets_pos_full['text'] = tweets_pos_full['text'].str.replace(' to ', '')
# tweets_pos_full['text'] = tweets_pos_full['text'].str.replace(' is ', '')
# tweets_pos_full['text'] = tweets_pos_full['text'].str.replace(' and ', '')
# tweets_pos_full['text'] = tweets_pos_full['text'].str.replace(' my ', '')
# tweets_pos_full['text'] = tweets_pos_full['text'].str.replace(' me ', '')
# tweets_pos_full['text'] = tweets_pos_full['text'].str.replace(' the ', '')
# tweets_pos_full['text'] = tweets_pos_full['text'].str.replace(' you ', '')
# tweets_pos_full['text'] = tweets_pos_full['text'].str.replace(' yourl ', '')
# tweets_pos_full['text'] = tweets_pos_full['text'].str.replace(' of ', '')
# tweets_pos_full['text'] = tweets_pos_full['text'].str.replace(' for ', '')
# tweets_pos_full['text'] = tweets_pos_full['text'].str.replace(' in ', '')
# tweets_pos_full['text'] = tweets_pos_full['text'].str.replace(' so ', '')
# tweets_pos_full['text'] = tweets_pos_full['text'].str.replace(' this ', '')
# tweets_pos_full['text'] = tweets_pos_full['text'].str.replace(' that ', '')
# #remove punctuaction
# tweets_pos_full['text'] = tweets_pos_full['text'].apply(lambda x: remove_punctuation(x))
# print("remove words!!")
# # remove words user, url, number
# tweets_pos_full['text'] = tweets_pos_full['text'].apply(lambda x: remove_words(x))
# print("Replace more leeters!!")
# # replace more letters haaaaaaaaaappy => haappy
# tweets_pos_full['text'] = tweets_pos_full['text'].apply(lambda x: replace_moreletters(x))
# tweets_pos_full['text'] = tweets_pos_full['text'].apply(lambda x: one_space(x))
    

In [ ]:

# #read data from .txt to preprocess
# tweets_neg_full = pd.read_fwf(DATA_FOLDER + '/train_neg_full.txt',  names=tweets_col_names,
#                               dtype=tweets_dtypes)

# #increase index to start at 1
# tweets_neg_full.index = tweets_neg_full.index +1
# #remove duplicates
# tweets_neg_full.drop_duplicates(inplace=True)
# test = tweets_neg_full.head(50)
# #apply pre_process funtioncs
# #remove stop words
# #     tweets_neg['text'] = tweets_neg['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
# print("emoji")
# # interpret emoji
# tweets_neg_full['text'] = tweets_neg_full['text'].apply(lambda x: interpret_emoji(x))
# print("split numbers")
# #separate number with letters  1234test123 =>  test 
# tweets_neg_full['text'] = tweets_neg_full['text'].apply(lambda x: split_number_text(x))
# print("hastag")
# #remove hashtag #
# tweets_neg_full['text'] = tweets_neg_full['text'].str.replace('#', '')
# print("ur your")
# #replace ur by your
# tweets_neg_full['text'] = tweets_neg_full['text'].str.replace('ur', 'your')
# print("expand contractions")
# #expand contractions don't => do not
# tweets_neg_full['text'] = tweets_neg_full['text'].apply(lambda x: expand_contractions(x))
# tweets_neg_full['text'] = tweets_neg_full['text'].str.replace(' ur ', '')
# tweets_neg_full['text'] = tweets_neg_full['text'].str.replace(' u ', '')
# tweets_neg_full['text'] = tweets_neg_full['text'].str.replace(' i ', '')
# tweets_neg_full['text'] = tweets_neg_full['text'].str.replace(' he ', '')
# tweets_neg_full['text'] = tweets_neg_full['text'].str.replace(' she ', '')
# tweets_neg_full['text'] = tweets_neg_full['text'].str.replace(' they ', '')
# tweets_neg_full['text'] = tweets_neg_full['text'].str.replace(' it ', '')
# tweets_neg_full['text'] = tweets_neg_full['text'].str.replace(' to ', '')
# tweets_neg_full['text'] = tweets_neg_full['text'].str.replace(' is ', '')
# tweets_neg_full['text'] = tweets_neg_full['text'].str.replace(' and ', '')
# tweets_neg_full['text'] = tweets_neg_full['text'].str.replace(' my ', '')
# tweets_neg_full['text'] = tweets_neg_full['text'].str.replace(' me ', '')
# tweets_neg_full['text'] = tweets_neg_full['text'].str.replace(' the ', '')
# tweets_neg_full['text'] = tweets_neg_full['text'].str.replace(' you ', '')
# tweets_neg_full['text'] = tweets_neg_full['text'].str.replace(' yourl ', '')
# tweets_neg_full['text'] = tweets_neg_full['text'].str.replace(' not ', '')
# tweets_neg_full['text'] = tweets_neg_full['text'].str.replace(' of ', '')
# tweets_neg_full['text'] = tweets_neg_full['text'].str.replace(' for ', '')
# tweets_neg_full['text'] = tweets_neg_full['text'].str.replace(' in ', '')
# tweets_neg_full['text'] = tweets_neg_full['text'].str.replace(' so ', '')
# tweets_neg_full['text'] = tweets_neg_full['text'].str.replace(' this ', '')
# tweets_neg_full['text'] = tweets_neg_full['text'].str.replace(' that ', '')
# print("punctuaction")
#     #remove punctuaction
# tweets_neg_full['text'] = tweets_neg_full['text'].apply(lambda x: remove_punctuation(x))
# print("remove words")
# # remove words user, url, number
# tweets_neg_full['text'] = tweets_neg_full['text'].apply(lambda x: remove_words(x))
# print("haaapy to haapy")
# # replace more letters haaaaaaaaaappy => haappy
# tweets_neg_full['text'] = tweets_neg_full['text'].apply(lambda x: replace_moreletters(x))
# tweets_neg_full['text'] = tweets_neg_full['text'].apply(lambda x: one_space(x))

# #remove stop words
  

In [ ]:
# combined = pd.concat([tweets_neg_full,tweets_pos_full])
# combined.head()


In [ ]:
# #remove most frequent words 
# freq_max_total = pd.Series(' '.join(combined['text']).split()).value_counts()[:10]
# freq_max_total = list(freq_max_total.index)

# #remove least frequent words
# freq_min_total = pd.Series(' '.join(combined['text']).split()).value_counts()[-10:]
# freq_min_total = list(freq_min_total.index)



In [ ]:
# freq_max_total

In [ ]:
# freq_min_total

In [ ]:
# freq_words = freq_max_total+ freq_min_total
# freq_words

In [ ]:
# #save the file to pickle
# print("Saving most frequent and lest frequent words")
# pickle_out = open("data/pre_processed/freq_words_10_withoutpronouns.pickle","wb")
# pickle.dump(freq_words, pickle_out)
# pickle_out.close()
# f = open("data/pre_processed/freq_words_10_withoutpronouns.txt", "w", encoding='utf-8')
# f.write("\n".join(map(lambda x: str(x), freq_words)))
# f.close()
# print("Saved!")

In [ ]:
# pickle_in = open("data/pre_processed/freq_words_5.pickle","rb")
# xxxxxx = pickle.load(pickle_in)
# print("Opening pickle")
# xxxxxx

In [ ]:
# tweets_pos = pd.read_fwf(DATA_FOLDER + '/train_pos.txt',  names=tweets_col_names,
#                               dtype=tweets_dtypes)
# # index start at 1
# tweets_pos.index = tweets_pos.index + 1
# #remove duplicates
# tweets_pos.drop_duplicates(inplace=True)

In [ ]:
# tweets_neg_full_pickle.str.contains('like').sum()
# tweets_pos.text.str.contains('like').sum()

In [ ]:

# tweets_pos.replace(freq_words_20,' ', inplace=True)
# tweets_pos.text.str.contains('like').sum()

In [ ]:
# tweets_pos['text'] = tweets_pos['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_words_20))
# tweets_pos.text.str.contains(' like ').sum()

In [ ]:
# freq_words_20

In [ ]:

    
# pickle_in = open("data/pre_processed/tweets_neg_freqwords20.pickle","rb")
# tweets_neg_freqwords20 = pickle.load(pickle_in)
# tweets_neg_freqwords20.str.contains('yourl').sum()



In [ ]:
# tweets_neg_freqwords20.apply(lambda x: " ".join(x for x in x.split() if x not in freq_words_20))
# tweets_neg_freqwords20.str.contains('yourl').sum()

In [ ]:
# test['text'] = test['text'].apply(lambda x: ''.join(TextBlob(x).correct()))
# test

In [ ]:
tweets_pos